# Задание 2 по курсу "Машинное обучение и анализ данных-1"

__Задача:__ обучить классификатор рукописных цифр на датасете MNIST.

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns

from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

## Загрузка данных с Kaggle:

Это можно сделать вручную или через kaggle-api. Краткая инструкция к Kaggle API:
* Установить kaggle: `pip install kaggle`.
* Зайти в `Your profile` -> вкладка `Account` -> раздел `API`.
* Нажать `Create new API Token`, положить файл по нужному адресу (он будет в подсказке).
* Выполнить команду `kaggle competitions download -c digit-recognizer`.
* Распаковать файлы из архива в текущую папку.

In [ ]:
!kaggle competitions download -c digit-recognizer
!unzip -f digit-recognizer.zip
!rm -f digit-recognizer.zip

Читаем данные:

In [2]:
train = pd.read_csv('./train.csv')
test = pd.read_csv('./test.csv')

print(train.shape, test.shape)

(42000, 785) (28000, 784)


In [3]:
X_train = train.drop(columns=['label']).copy()
y_train = train.label

X_test = test.copy()

Выберем 5000 наблюдений из исходной выборки для того, чтобы быстрее гонять эксперименты.

In [4]:
subsample_size = 5000

np.random.seed(42)
subsample_index = np.random.choice(train.shape[0], subsample_size)

In [5]:
Xs_train = X_train.iloc[subsample_index]
ys_train = y_train.iloc[subsample_index]

## Учим логистическую регрессию:

In [6]:
%%time
scaler = StandardScaler()
logreg = LogisticRegression(max_iter=1024)

model = Pipeline([
    ('scaler', scaler),
    ('logreg', logreg)
])
model.fit(Xs_train, ys_train)

CPU times: user 26.4 s, sys: 1.74 s, total: 28.2 s
Wall time: 3.66 s


Pipeline(steps=[('scaler', StandardScaler()),
                ('logreg', LogisticRegression(max_iter=1024))])

## Готовим submission и отправляем в Kaggle

In [7]:
submission = pd.DataFrame({
    'ImageId': np.arange(1, X_test.shape[0] + 1),
    'Label': model.predict(X_test)
})
submission.head()

,ImageId,Label
0,1,2
1,2,0
2,3,9
3,4,4
4,5,3


In [8]:
submission.to_csv('submission.csv', index=False)

Промежуточные сабмиты именовать по правилу: `MLDA, <ФИО>, <краткое описание модели>`.

У итогового сабмита добавить слово `финальный` в конце.

In [9]:
!kaggle competitions submit -c digit-recognizer -f submission.csv -m "<имя сабмита>"

100%|████████████████████████████████████████| 208k/208k [00:04<00:00, 46.3kB/s]
Successfully submitted to Digit Recognizer